In [1]:
# %load_ext autoreload
# %autoreload 2

In [1]:
import sys
import os
sys.path.append('perteval')
sys.path.append('uq_score')

In [2]:
import json
from utils import get_requests, get_results
from perteval.perteval_end_to_end import get_perteval_results
# from uq_score.uq_end_to_end import run_uq
from llm_uncertainty_bench.uq_end_to_end import run_uq_benchmark
import datetime
import perteval.transition_analysis as tas
import time
import subprocess

In [3]:
# try killing old processes that utilize the gpu
try:
    subprocess.run(["nvidia-smi | grep 'python' | awk '{ print $5 }' | xargs -n1 kill -9"], shell=True)
except:
    pass

In [4]:
if len(sys.argv) == 1 or sys.argv[1] == '-f':
    # -f is to avoid the case that this runs as ipynb
    all_request_models = get_requests('cc4718/requests')
    all_result_models = get_results('cc4718/results')
    models_todo = set(all_request_models) - set(all_result_models)
    # dataset can be full/sample
    dataset = 'full'
else:
    # huggingface models
    models_todo = [sys.argv[1]]
    # dataset full/sample
    dataset = sys.argv[2]
print(models_todo)
print(dataset)

Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

/u/modelfactory/.cache/huggingface/hub/datasets--cc4718--requests/snapshots/ed8a000faff905923cadd18a9229d2d1b5036768


Fetching 26 files:   0%|          | 0/26 [00:00<?, ?it/s]

/u/modelfactory/.cache/huggingface/hub/datasets--cc4718--results/snapshots/2b7dae5c279fc817670edd2b2eb2774a97b0e415/demo-leaderboard/gpt2-demo
{'ibm-granite/granite-3.1-8b-instruct'}
full


In [ ]:
for model_name in models_todo:
    # perteval
    original_log_path = get_perteval_results(model_name, mode='original', cot='cot_standard', dataset=dataset)
    print('Finished perteval on original data')
    # wait a few seconds to kill vllm to spin up a new
    time.sleep(10)
    perturb_log_path = get_perteval_results(model_name, mode='perturb', cot='cot_standard', dataset=dataset)
    print('Finished perteval on perturbed data')
    original, perturb, consist = tas.transition_analysis(original_log_path, perturb_log_path, subjects=["failure_mode_sensor_analysis"])
    asset_scores = tas.get_record_id_for_correct_answer(original_log_path, dimention='asset_name')
    relevancy_scores = tas.get_record_id_for_correct_answer(original_log_path, dimention='relevancy')
    # uq on the original dataset
    # dataset_path = 'data/fmsr'
    # accuracy, nll_loss_avg, ece_score_avg = run_uq(model_name=model_name, dataset_path=dataset_path)
    # uq bench
    print('Running LLM Uncertainty Bench')
    uq_scores = run_uq_benchmark(model_name, prompt_type='chat', dataset=dataset)
    result_dict = {
        "config": {
            "model_dtype": "torch.bfloat16", 
            "model_name": model_name,
            "model_sha": "main"
        },
        "results": {
            "acc_overall": {
                "acc": original
            },
            "acc_sel": {
                "acc_sel": relevancy_scores['relevant_sensor_for_failure_mode']
            },
            "acc_el": {
                "acc_el": relevancy_scores['irrelevant_sensor_for_failure_mode']
            },
            "acc_perturb": {
                "perturb_score": perturb
            },
            "score_consistency": {
                "consist_score": consist
            }
        },
    }
    for k, v in uq_scores.items():
        result_dict['results'][k] = {k: v}
    for asset in asset_scores:
        if not asset:
            asset_lower = 'other'
        else:
            asset_lower = asset.lower().replace(' ', '_')
        result_dict['results'][f'acc_{asset_lower}'] = {f'acc_{asset_lower}': asset_scores[asset]}
    out_model_name = model_name.replace('/', '--')
    out_fname = f'results/demo-leaderboard/gpt2-demo/results_{out_model_name}.json'
    with open(out_fname, 'w') as f:
        f.write(json.dumps(result_dict))

INFO:root:Log path: ./log/fmsr_filtered_data_full_2025-06-05T11:28:29.jsonl
INFO:root:prompt: Please select the correct option(s) from the following options given the question. To solve the problem, follow the Let's think Step by Step reasoning strategy.
Question: For electric motor, if a failure event rotor windings fault occurs, which sensor out of the choices is the most relevant sensor regarding the occurrence of the failure event?
Options:
A partial discharge
B resistance
C oil debris
D current
E voltage
{"option_a": "<Reasoning for option a>", "option_b": "<Reasoning for option b>", ..., "answer": <the list of selected option, e.g., ["A", "B", "C", "D", "E"]>}
Your output in a single line:
INFO:root:prompt: Please select the correct option(s) from the following options given the question. To solve the problem, follow the Let's think Step by Step reasoning strategy.
Question: When dealing with bearing damage in electric generator, which sensor among the choices has the highest rel

# of samples = 2667
0
0
0
0
0
0
0
0


INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
ERROR:root:original_llm_answer = 
INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
ERROR:root:original_llm_answer = 
ERROR:root:original_llm_answer = 
ERROR:root:original_llm_answer = 
ERROR:root:original_llm_answer = 
ERROR:root:Traceback (most recent call last):
  File "/dccstor/dcpfactory/test/FailureSensorIQ/perteval/KPPerturbation.py", line 290, in get_mcq_llm_answer
    original_response = llm.listen_and_response(prompt)
  File "/dccstor/dcpfactory/test/FailureSensorIQ/perteval/GeneralLLM.py", line 269, in listen_and_response
    response = self.client

1
1
1
1
1


INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
ERROR:root:original_llm_answer = 
INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
ERROR:root:original_llm_answer = 
INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
ERROR:root:original_llm_answer = 
ERROR:root:Traceback (most recent call last):
  File "/dccstor/dcpfactory/test/FailureSensorIQ/perteval/KPPerturbation.py", line 286, in get_mcq_llm_answer
    original_response = llm.listen_and_response(
  File "/dccstor/dcpfactory/test/FailureSensorIQ/perteval/GeneralLLM.py", line 269, in listen_and_response
    response = self.client.chat.completions.create(
  File "/u/modelfactory/.conda/envs/lab/lib/pytho

...

64
64
64
64
64


INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
ERROR:root:original_llm_answer = 
ERROR:root:original_llm_answer = 
INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
ERROR:root:Traceback (most recent call last):
  File "/dccstor/dcpfactory/test/FailureSensorIQ/perteval/KPPerturbation.py", line 286, in get_mcq_llm_answer
    original_response = llm.listen_and_response(
  File "/dccstor/dcpfactory/test/FailureSensorIQ/perteval/GeneralLLM.py", line 269, in listen_and_response
    response = self.client.chat.completions.create(
  File "/u/modelfactory/.conda/envs/lab/lib/python3.10/site-packages/openai/_utils/

65
65
65
65
65


INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
ERROR:root:original_llm_answer = 
INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 404 Not Found"
ERROR:root:original_llm_answer = 
ERROR:root:original_llm_answer = 
ERROR:root:original_llm_answer = 
ERROR:root:original_llm_answer = 
ERROR:root:Traceback (most recent call last):
  File "/dccstor/dcpfactory/test/FailureSensorIQ/perteval/KPPerturbation.py", line 286, in get_mcq_llm_answer
    original_response = llm.listen_and_response(
  File "/dccstor/dcpfactory/test/FailureSensorIQ/perteval/GeneralLLM.py", line 269, in listen_and_response
    response = self.client.chat.c

66
66
66
66
66


In [16]:
# !nvidia-smi | grep 'python' | awk '{ print $5 }' | xargs -n1 kill -9

kill: not enough arguments
